In [7]:
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm
import random
import functools

import pomegranate
from pomegranate import HiddenMarkovModel ,State
from pomegranate.distributions import IndependentComponentsDistribution
from pomegranate.distributions import NormalDistribution

import copy

In [2]:
n_dim_of_chain=8
n_of_chains = 5


# reload the meta network

In [3]:
import pickle as pkl
with open("markov_network",'rb') as f : 
    state_comb_to_walks_comb_dict = pkl.load(f)

# generate_data

In [4]:
n_of_states_in_meta_network = len(state_comb_to_walks_comb_dict.keys())
size_of_possible_rw = 30
number_of_seqs = 500

In [5]:
def pick_random_next_stage(_possible_next_steps,state_comb_to_walks_comb_dict) : 
    if len(_possible_next_steps) == 0 :
        return None
    
    first_pick = random.choice(_possible_next_steps)
    if first_pick in all_possible_states :
        return first_pick
    _possible_next_steps.remove(first_pick)
    return pick_random_next_stage(_possible_next_steps,state_comb_to_walks_comb_dict)
    

In [8]:
all_possible_states = list(state_comb_to_walks_comb_dict.keys())
seqs = [] 
for i in range(number_of_seqs):
    seq = []
    random_state_idx = random.randint(0,n_of_states_in_meta_network)
    curr_random_state = all_possible_states[random_state_idx]
    seq.append(curr_random_state)
    
    for j in range(size_of_possible_rw) : 
        possible_next_steps = copy.copy(state_comb_to_walks_comb_dict[curr_random_state])
        curr_random_state = pick_random_next_stage(possible_next_steps,state_comb_to_walks_comb_dict)
        
        if curr_random_state is None : 
            random_state_idx = random.randint(0,n_of_states_in_meta_network)
            curr_random_state = all_possible_states[random_state_idx]
            
        seq.append(curr_random_state)
        
    seqs.append(seq)
    

KeyboardInterrupt: 

In [ ]:
def return_multi_hot(state_set) : 
    multi_hot_vector_state = np.zeros((n_of_chains*n_dim_of_chain,1))
    multi_hot_vector_state[list(state_set)] = 1
    return multi_hot_vector_state.T[0]

def return_multi_hot_vectors(vectors) :
    return np.array([return_multi_hot(vector) for vector in vectors])
    
sampled_seqs = [return_multi_hot_vectors(random.choices(s,k=4)) for s in seqs]

In [ ]:
# with open("sampled_seqs.pkl",'wb') as f : 
#     pkl.dump(sampled_seqs,f)

In [ ]:
sampled_seqs[1].shape